In [6]:
import xlrd
import xlwt
import pandas as pd

In [18]:
def DataArrage():
    # 一、数据预处理
    # 1、Excel表格读的部分：
    excel = xlrd.open_workbook('01.蔬菜价格.xls')  # 读入本地名为‘o1.蔬菜价格.xls’的Excel表格
    excelsheet = excel.sheets()[0]                # 获取excel表格中的第一个sheet工作表
    # 2、Excel表格写的部分：
    newexcel = xlwt.Workbook(encoding='utf-8')   # 创建一个Excel对象,编码格式采用utf-8
    # 创建一个表名为“蔬菜价格关联分析”的工作表，cell_overwrite_ok=True表示可覆盖之前已操作的单元格
    newsheet = newexcel.add_sheet('蔬菜价格关联分析', cell_overwrite_ok=True)
    # 3、设置表格样式，这里主要用于设置日期的格式
    datastyle = xlwt.XFStyle()                          # 创建一个样式对象，初始化样式
    datastyle.num_format_str = 'M/D/YY'                 # 设置日期格式
    # 4、表格中有的空的位置不是真的为空，这里获取一个这样的值，待会用于特判
    tmp = excelsheet.cell_value(5, 2)
    # 二、正式开始处理数据，主要思路：从旧表中读出写入新表
    ri = 2  # 用于读原始第ri行
    rj = 0  # 用于读原始第rj列
    wi = 0  # 用于写入表格的第wi行
    wj = 1  # 用于写入表格的第wj列
    # 创建新表的第一行，包括日期和各个菜的菜名
    newsheet.write(0, 0, excelsheet.cell_value(0, 0))  # 将旧表的（0，0）位置的日期写到新表的（0，0）位置
    # 1、先读取蔬菜列和禽肉类的列名，暂时先不处理禽肉类的一个K位置为空值的特殊情况
    while ri < 42:
        rj = 1  # 蔬菜在第1列,有40种,肉类在第3列，有23种
        newsheet.write(0, wj, excelsheet.cell_value(ri, rj))
        newsheet.write(0, wj + 40, excelsheet.cell_value(ri, rj + 2))
        ri = ri + 1
        wj = wj + 1
    # 2、先整理蔬菜的价格
    ri = 42  # 遍历原始第ri行
    rj = 2  # 遍历原始第rj列
    wi = 0  # 写入表格的第wi行
    wj = 1  # 写入表格的第wj行
    while ri < 22320:
        if ri % 47 >= 42:
            ri = ri + 6
            wi = wi + 1
            newsheet.write(wi, 0, excelsheet.cell_value(ri, 0), datastyle)
        else:
            if excelsheet.cell_value(ri, rj) != tmp:
                newsheet.write(wi, wj, excelsheet.cell_value(ri, rj))
            wj = wj % 40 + 1
        ri = ri + 1
    # 3、再整理禽肉类的价格
    ri = 26  # 遍历原始第ri行
    rj = 4  # 遍历原始第4列
    wi = 0  # 写入表格的第wi行
    wj = 41  # 写入表格的第wj行，从第41列开始
    while ri < 22320:
        if ri % 47 >= 26:
            ri = ri + 22
            wi = wi + 1
            wj = 41
        else:
            if excelsheet.cell_value(ri, rj) != tmp:
                newsheet.write(wi, wj, excelsheet.cell_value(ri, rj))
            wj = wj % (41 + 24) + 1
        ri = ri + 1
    newexcel.save('Transition.xls')  # 生成处理后的表格
###################################################################################
    # header = None用于不让蔬菜名那一行作为“表头”，便于之后步骤的数据获取
    data = pd.read_excel('Transition.xls', header=None)
    # 删除第60列空值列，axis=1表示纵向，inplace=True表示删除操作对原数据生效
    data.drop(labels=[60], axis=1, inplace=True)
    ri = 1
    while ri < data.shape[0] - 1:
        rj = 1
        while rj < data.shape[1]:
            if (len(str(data.values[ri, rj])) == 0 or len(str(data.values[ri + 1, rj])) == 0):
                data.iloc[ri, rj] = 0
            else:
                d = float(data.values[ri, rj]) - float(data.values[ri + 1, rj]) 
         # 上-下 今天-昨天  正为涨，负为跌
                if d > 0:
                    data.iloc[ri, rj] = 'up'
                elif d < 0:
                    data.iloc[ri, rj] = 'down'
                else:
                    data.iloc[ri, rj] = 0
            rj = rj + 1
        ri = ri + 1
    data.drop(labels=[data.shape[0] - 1], axis=0, inplace=True)  # 删除最后一行
    data.to_excel("Finished_version.xlsx")


In [19]:
DataArrage()

In [20]:
import pandas as pd
from openpyxl import load_workbook
from efficient_apriori import apriori

data=pd.read_excel('Finished_version.xlsx',header=None)
# print(data)
data_list = []
a =int(data.shape[1])-1
b =int(data.shape[0])-1
for j in range(2,b+1):#列数64 
    data_up_list = []
    for i in range(2,a+1):#行数474
        if(data[i][j]=='up'):
            # print(data[i][1],'涨')
            data_up_list.append(data[i][1]+'1')
        elif(data[i][j]=='down'):
            # print(data[i][1],'跌')
            data_up_list.append(data[i][1]+'1')
    # print(data_up_list)
    data_list.append(data_up_list)
# print(data_list)   


# 设置数据集
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(data_list, min_support=0.23,  min_confidence=0.5)
print(itemsets)
print(rules)

{1: {('大白菜1',): 208, ('葱1',): 155, ('菠菜1',): 204, ('韭菜1',): 164, ('芸豆1',): 204, ('茄子1',): 184, ('青椒1',): 166, ('茭瓜1',): 162, ('菜花1',): 203, ('鲜蘑菇1',): 176, ('茼蒿1',): 162, ('油菜1',): 175, ('黄瓜1',): 191, ('芹菜1',): 141, ('大头菜1',): 179, ('西红柿1',): 177, ('尖椒1',): 174, ('香菜1',): 118}, 2: {('大头菜1', '大白菜1'): 109, ('大白菜1', '芸豆1'): 122, ('大白菜1', '茄子1'): 111, ('大白菜1', '菜花1'): 115, ('油菜1', '芸豆1'): 111, ('芸豆1', '茄子1'): 111, ('芸豆1', '菜花1'): 120, ('芸豆1', '菠菜1'): 110}}
[{大白菜1} -> {大头菜1}, {大头菜1} -> {大白菜1}, {芸豆1} -> {大白菜1}, {大白菜1} -> {芸豆1}, {茄子1} -> {大白菜1}, {大白菜1} -> {茄子1}, {菜花1} -> {大白菜1}, {大白菜1} -> {菜花1}, {芸豆1} -> {油菜1}, {油菜1} -> {芸豆1}, {茄子1} -> {芸豆1}, {芸豆1} -> {茄子1}, {菜花1} -> {芸豆1}, {芸豆1} -> {菜花1}, {菠菜1} -> {芸豆1}, {芸豆1} -> {菠菜1}]


In [21]:
excel = xlrd.open_workbook('veg.xlsx')  # 读入本地名为‘o1.蔬菜价格.xls’的Excel表格
excelsheet = excel.sheets()[0]  

In [14]:
dataSet = []
rowSet = []
ri = 2
rj = 0
while ri <= 474:
    while rj <= 39:
        tmp = excelsheet.cell_value(ri,rj)
        if  tmp != 0:
            rowSet.append(excelsheet.cell_value(1,rj))
        rj = rj+1
    dataSet.append(rowSet)
    rowSet = []
    ri = ri + 1
    rj = 0

In [53]:
itemsets, rules = apriori(dataSet, min_support=0.25,  min_confidence=0.5)
print(itemsets)
print(rules)

{1: {('大白菜',): 208, ('葱',): 155, ('菠菜',): 204, ('韭菜',): 164, ('芸豆',): 204, ('茄子',): 184, ('青椒',): 166, ('茭瓜',): 162, ('菜花',): 203, ('鲜蘑菇',): 176, ('茼蒿',): 162, ('油菜',): 175, ('黄瓜',): 191, ('芹菜',): 141, ('大头菜',): 179, ('西红柿',): 177, ('尖椒',): 174}, 2: {('大白菜', '芸豆'): 122, ('芸豆', '菜花'): 120}}
[{芸豆} -> {大白菜}, {大白菜} -> {芸豆}, {菜花} -> {芸豆}, {芸豆} -> {菜花}]


In [15]:
excel = xlrd.open_workbook('meat.xlsx')  # 读入本地名为‘o1.蔬菜价格.xls’的Excel表格
excelsheet = excel.sheets()[0]  

In [16]:
dataSet = []
rowSet = []
ri = 2
rj = 0
while ri <= 473:
    fg = False
    while rj <= 22:
        tmp = excelsheet.cell_value(ri,rj)
        if  tmp != 0:
            fg = True
            rowSet.append(excelsheet.cell_value(0,rj))
        rj = rj+1
    if fg: 
        dataSet.append(rowSet)
    rowSet = []
    ri = ri + 1
    rj = 0

In [17]:
itemsets, rules = apriori(dataSet, min_support=0.1,  min_confidence=0.5)
print(itemsets)
print(rules)

{1: {('鸡蛋',): 60, ('鸡翅',): 11, ('鸡胸肉',): 12, ('猪肝',): 9, ('猪蹄',): 10, ('精猪肉',): 10, ('白条肉',): 14, ('西装鸡',): 11, ('鸡大腿',): 11, ('翅根',): 15, ('翅中',): 15, ('精猪肚',): 9}, 2: {('白条肉', '精猪肉'): 9, ('白条肉', '翅中'): 9, ('精猪肉', '翅中'): 9, ('精猪肉', '鸡大腿'): 9, ('翅中', '翅根'): 11, ('翅根', '西装鸡'): 9, ('西装鸡', '鸡大腿'): 9}}
[{精猪肉} -> {白条肉}, {白条肉} -> {精猪肉}, {翅中} -> {白条肉}, {白条肉} -> {翅中}, {翅中} -> {精猪肉}, {精猪肉} -> {翅中}, {鸡大腿} -> {精猪肉}, {精猪肉} -> {鸡大腿}, {翅根} -> {翅中}, {翅中} -> {翅根}, {西装鸡} -> {翅根}, {翅根} -> {西装鸡}, {鸡大腿} -> {西装鸡}, {西装鸡} -> {鸡大腿}]
